# Models Evaluation

## Introduction
In this notebook, optimal hyperparameters will be selected and the performance of both models will be evaluated.

### Imports
The analysis commences with the necessary imports.

In [ ]:
import os
import pandas as pd
import numpy as np

import sys
from pathlib import Path

project_root = Path.cwd()
while not (project_root / "src").exists():
    project_root = project_root.parent

sys.path.append(str(project_root / "src"))

from model_selection import grid_search_cv
from models import SVM, LogisticRegression

### Data Loading
The data will be loaded.

In [ ]:
X_train_df = pd.read_csv('../data/processed/X_train.csv')
y_train_df = pd.read_csv('../data/processed/y_train.csv')
X_test_df = pd.read_csv('../data/processed/X_test.csv')
y_test_df = pd.read_csv('../data/processed/y_test.csv')

y_train = np.where(y_train_df['quality'] >= 6, 1, -1)
y_test = np.where(y_test_df['quality'] >= 6, 1, -1)

X_train = X_train_df.to_numpy()
X_test = X_test_df.to_numpy()

## Hyperparameter Tuning
To identify optimal hyperparameters, multiple rounds of grid search are required to thoroughly explore all possible parameter combinations.

### SVM
For SVMs, two primary parameters require optimization: the number of iterations (*n_iters*) and the regularization parameter lambda (*lambda_param*). Typically, the number of folds ranges between 5 to 10; however, given our computational capacity, we can extend this to 100 folds without exceeding three minutes of processing time, thereby approximating Leave-One-Out validation and achieving a more robust validation framework.

In [ ]:
svm_param_grid = {
        'n_iters': [1000, 2000, 3000, 4000, 5000, 6000],
        'lambda_param' : [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
    }

svm_best_params, svm_best_metrics = grid_search_cv(SVM, svm_param_grid, X_train, y_train, 100)
print(f'SVM best parameter: {svm_best_params}')
print(f'SVM best metrics: {svm_best_metrics}')

The optimal hyperparameters identified are *n_iters: 5000* and *lambda_param: 0.1*. A refined search will now be conducted within the neighborhood of these parameters.

In [ ]:
svm_param_grid = {
        'n_iters': [4500, 4750, 5000, 5250, 5500],
        'lambda_param' : [5e-1, 3e-1, 1e-1, 9e-2, 7e-2]
    }

svm_best_params, svm_best_metrics = grid_search_cv(SVM, svm_param_grid, X_train, y_train, 100)
print(f'SVM best parameter: {svm_best_params}')
print(f'SVM best metrics: {svm_best_metrics}')

The optimal hyperparameters identified now are *n_iters: 5000* and *lambda_param: 0.07*. Let's continue with the Logistic Regression.